In [6]:
import numpy as np
import pandas as pd
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
#from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import pycaret
from pycaret.classification import *

from pickle import dump
from pickle import load

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
test_df = pd.read_csv('test.csv')
test_df.drop('Id',inplace=True,axis=1)
test_df.drop(columns=['Soil_Type15','Soil_Type7'],inplace=True)

In [9]:
categorical_var = []
continous_var = []
for col in test_df.columns:
    if test_df[col].nunique() == 2:
        categorical_var.append(col)
    else:
        if col != 'Cover_Type':
            continous_var.append(col)

In [10]:
for col in categorical_var:
    test_df[col] = test_df[col].astype('bool')

In [11]:
test_df['Hillshade_Noon'].clip(lower=0,upper=255,inplace=True)
test_df['Hillshade_9am'].clip(lower=0,upper=255,inplace=True)
test_df['Hillshade_3pm'].clip(lower=0,upper=255,inplace=True)

test_df["Aspect"][test_df["Aspect"] < 0] += 360
test_df["Aspect"][test_df["Aspect"] > 359] -= 360

test_df["ecldn_dist_hydrlgy"] = (test_df["Horizontal_Distance_To_Hydrology"]**2 + test_df["Vertical_Distance_To_Hydrology"]**2)**0.5

In [12]:
loaded_model = load_model('Final RF Model')

Transformation Pipeline and Model Successfully Loaded


In [13]:
predictions = predict_model(loaded_model, data=test_df, raw_score=True)

In [14]:
predictions.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,ecldn_dist_hydrlgy,Label,Score_0,Score_1,Score_2,Score_3,Score_4,Score_5
0,2763,78,20,377,88,3104,218,213,195,1931,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,387.134344,2,0.0005,0.9993,0.0002,0.0,0.0000,0.0
1,2826,153,11,264,39,295,219,238,148,2557,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,266.865134,2,0.0246,0.9751,0.0002,0.0,0.0000,0.0
2,2948,57,19,56,44,852,202,217,163,1803,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,71.217975,2,0.0052,0.9946,0.0001,0.0,0.0001,0.0
3,2926,119,6,158,134,2136,234,240,142,857,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,207.171427,2,0.0002,0.9892,0.0105,0.0,0.0000,0.0
4,2690,10,4,38,108,3589,213,221,229,431,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,114.490174,2,0.0006,0.9991,0.0003,0.0,0.0001,0.0


In [16]:
dataset_id = pd.read_csv('test.csv')
dataset_id.reset_index(inplace=True)
dataset_id = dataset_id.loc[:,'Id']
my_submission = pd.DataFrame({'Id': dataset_id, 'Cover_Type': predictions['Label']})
my_submission.to_csv('submission.csv', index=False)